In [136]:
import numpy as np
import pandas as pd
import numpy.typing as npt
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json
import itertools

In [137]:
# defining constants
TEST_SIZE: np.float_ = .3
RANDOM_STATE: np.int_ = 42

In [138]:
def plot_number(label: np.int_, pixels: npt.NDArray[np.int_]) -> None:
    plt.title(f"True label: {int(label)}")
    plt.imshow(np.reshape(pixels, (28, 28)), cmap="gray")
    plt.axis("off")
    plt.show()

In [139]:
# importing and handling data
data: pd.DataFrame = pd.read_csv("data/input/data_tp1", header=None)
data.iloc[:, data.columns != 0] = data.iloc[:, data.columns != 0] / 255
data.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
def MNIST_MLP(data: npt.NDArray[np.int_], hidden_layer_size: np.int_, batch_size: np.int_, learning_rate: np.int_) -> dict:

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(784,)),
        tf.keras.layers.Dense(hidden_layer_size, activation="sigmoid"),
        tf.keras.layers.Dense(10)
    ])

    input_data: npt.NDArray[np.int_] = data[:, 1:]
    labels: npt.NDArray[np.int_] = data[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    metrics = ["accuracy"]
    
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    epochs: np.int_ = 10

    model_history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=0)

    y_pred: npt.NDArray[np.int_] = model.predict(X_test, batch_size=batch_size, verbose=0)
    y_pred = y_pred.argmax(axis=-1)

    accuracy_score: np.float_ = sklearn.metrics.accuracy_score(y_test, y_pred)
    precision_score: npt.NDArray[np.float_] = sklearn.metrics.precision_score(y_test, y_pred, average=None, zero_division=0)
    recall_score: npt.NDArray[np.float_] = sklearn.metrics.recall_score(y_test, y_pred, average=None, zero_division=0)
    f1_score: npt.NDArray[np.float_] = sklearn.metrics.f1_score(y_test, y_pred, average=None)
    confusion_matrix: npt.NDArray[np.int_] = sklearn.metrics.confusion_matrix(y_test, y_pred)

    run_info: dict = {
        "accuracy_score": accuracy_score,
        "precision_score": precision_score.tolist(),
        "recall_score": recall_score.tolist(),
        "f1_score": f1_score.tolist(),
        "confusion_matrix": confusion_matrix.tolist(),
        "random_state_seed": RANDOM_STATE,
        "test_size": TEST_SIZE,
        "batch_size": batch_size,
        "learning_rate": learning_rate
    }

    return run_info

In [149]:
hidden_layer_sizes: npt.NDArray[np.int_] = np.array([(25,), (50,), (100,)])
batch_sizes: list[int] = [1, 20, 50, 3500]
lerning_rates: npt.NDArray[np.float_] = np.array([0.5, 1.0, 10.0])

configurations: list = list(itertools.product(hidden_layer_sizes, batch_sizes, lerning_rates))

run_infos: list[dict] = [MNIST_MLP(data=data.to_numpy(), hidden_layer_size=a, batch_size=b, learning_rate=c) for a, b, c in configurations]

In [150]:
with open("data/results.json", "a") as file:
    json.dump([run_info for run_info in run_infos], file, indent=4)

In [154]:
df = pd.read_json("data/results.json")
df.nsmallest(n=10, columns=["accuracy_score"])

,accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,random_state_seed,test_size,batch_size,learning_rate
26,0.100667,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.100...","[0.0, 0.0, 0.0, 0.0, 0.0059171597633130004, 0....","[0.0, 0.0, 0.0, 0.0, 0.011764705882352, 0.0, 0...","[[0, 0, 0, 0, 0, 0, 0, 0, 125, 0], [0, 0, 0, 0...",42,0.3,1,10.0
14,0.132667,"[0.0, 1.0, 0.095899930507296, 0.0, 0.0, 0.0, 0...","[0.0, 0.37423312883435506, 1.0, 0.0, 0.0, 0.0,...","[0.0, 0.5446428571428571, 0.175015852885225, 0...","[[0, 0, 125, 0, 0, 0, 0, 0, 0, 0], [0, 61, 102...",42,0.3,1,10.0
35,0.137333,"[0.0, 0.875, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.386503067484662, 0.007246376811594001,...","[0.0, 0.536170212765957, 0.014388489208633, 0....","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 125], [0, 63, 0, ...",42,0.3,3500,10.0
2,0.187333,"[0.0, 0.9025974025974021, 0.0, 0.1054977711738...","[0.0, 0.8527607361963191, 0.0, 0.9930069930069...","[0.0, 0.8769716088328071, 0.0, 0.1907320349227...","[[0, 0, 0, 125, 0, 0, 0, 0, 0, 0], [0, 139, 0,...",42,0.3,1,10.0
11,0.225333,"[0.0, 0.9666666666666661, 0.0, 0.1468459152016...","[0.0, 0.8895705521472391, 0.0, 0.9930069930069...","[0.0, 0.9265175718849841, 0.0, 0.2558558558558...","[[0, 0, 0, 98, 0, 6, 0, 0, 16, 5], [0, 145, 0,...",42,0.3,3500,10.0
23,0.304000,"[0.265306122448979, 0.5536332179930791, 0.6000...","[0.10400000000000001, 0.9815950920245391, 0.26...","[0.14942528735632102, 0.707964601769911, 0.363...","[[13, 0, 4, 105, 0, 0, 0, 3, 0, 0], [0, 160, 0...",42,0.3,3500,10.0
25,0.429333,"[0.9636363636363631, 0.9006622516556291, 1.0, ...","[0.424, 0.8343558282208581, 0.0144927536231880...","[0.588888888888888, 0.8662420382165601, 0.0285...","[[53, 0, 0, 26, 0, 12, 34, 0, 0, 0], [0, 136, ...",42,0.3,1,1.0
8,0.439333,"[0.6740331491712701, 0.8315789473684211, 0.451...","[0.976, 0.9693251533742331, 0.8768115942028981...","[0.7973856209150321, 0.895184135977337, 0.5960...","[[122, 0, 0, 0, 0, 3, 0, 0, 0, 0], [0, 158, 1,...",42,0.3,50,10.0
34,0.494667,"[0.7851851851851851, 0.404580152671755, 0.3324...","[0.848, 0.9754601226993861, 0.884057971014492,...","[0.815384615384615, 0.571942446043165, 0.48316...","[[106, 1, 15, 0, 1, 1, 1, 0, 0, 0], [0, 159, 4...",42,0.3,3500,1.0
9,0.498667,"[0.6205128205128201, 0.4, 0.49275362318840504,...","[0.968, 0.9815950920245391, 0.7391304347826081...","[0.7562500000000001, 0.568383658969804, 0.5913...","[[121, 0, 2, 1, 0, 0, 0, 0, 0, 1], [0, 160, 1,...",42,0.3,3500,0.5


De acordo com https://www.tensorflow.org/guide/core/mlp_core, Next, rescale the data so that the pixel values of [0,255] fit into the range of [0,1]. This step ensures that the input pixels have similar distributions and helps with training convergence.